In [27]:
# Importing Pandas
import pandas as pd
import plotly.express as px
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
# Reading csv as dataframe
g20_index_data = pd.read_csv("UNION.csv")
g20_covid = pd.read_csv("g20_covid.csv")


In [28]:
g20_index_data

,date,Country,USD
0,2018-01-01 0:00,Argentina,0.000000
1,2018-01-02 0:00,Argentina,1671.991824
2,2018-01-03 0:00,Argentina,1711.535940
3,2018-01-04 0:00,Argentina,1732.425964
4,2018-01-05 0:00,Argentina,1729.343505
...,...,...,...
35795,2022-11-21 0:00,United States,0.000000
35796,2022-11-22 0:00,United States,0.000000
35797,2022-11-23 0:00,United States,0.000000
35798,2022-11-24 0:00,United States,0.000000


In [29]:
g20_index_data['USD'] = g20_index_data['USD'].replace(0, None)
g20_index_data.isna().sum()

date           0
Country        0
USD        11806
dtype: int64

In [30]:
g20_index_data.ffill(inplace=True)

In [31]:
# Cleaning Dataset

# Cleaning the date column to have only dates and not timestamps
g20_covid['date'] = pd.to_datetime(g20_covid['date'], format="%Y/%m/%d")
g20_index_data['date'] = pd.to_datetime(g20_index_data['date'], format="%Y/%m/%d")

# Replacing the missing values of the indexes with the last seen value
g20_index_data['USD'] = g20_index_data['USD'].replace(0, None)
g20_index_data.ffill(inplace=True)

# As we only have COVID data from february of 2020 we are going to cut g20_indexes to start the data from february of 2018
g20_index_data = g20_index_data[g20_index_data['date'] >= pd.to_datetime('2018-02-01', format="%Y/%m/%d")]

# Feature Engineering
# Creating a difference column for Index Data
# g20_index_data['difference'] = g20_index_data['USD'].diff()

# Creating a percentage of change for Index Data
# g20_index_data['pct_change'] = g20_index_data['USD'].pct_change()

# Creating a specific column from each part of the Date column
g20_covid['year'] = g20_covid['date'].dt.year
g20_covid['month'] = g20_covid['date'].dt.month
g20_covid['day'] = g20_covid['date'].dt.day_of_year
g20_covid['year_month'] = g20_covid['date'].dt.to_period('M')
g20_index_data['year'] = g20_index_data['date'].dt.year
g20_index_data['month'] = g20_index_data['date'].dt.month
g20_index_data['day'] = g20_index_data['date'].dt.day_of_year
g20_index_data['year_month'] = g20_index_data['date'].dt.to_period('M')

# Creating a return and pct_change column
g20_index_data = g20_index_data.sort_values(by=['Country', 'date'])

g20_index_data['return'] = 1
g20_index_data['pct_change'] = 1

for country in list(g20_index_data['Country'].unique()):
    g20_index_data.loc[g20_index_data['Country'] == country, 'return'] = g20_index_data.loc[g20_index_data['Country'] == country, 'USD'].diff()
    g20_index_data.loc[g20_index_data['Country'] == country, 'pct_change'] = g20_index_data.loc[g20_index_data['Country'] == country, 'USD'].pct_change()*100

# Creating a Continent Column
g20_index_data['Continent'] = g20_index_data['Country'].map({'Argentina':'South America', 'Australia':'Ocenia', 'Brazil':'South America', 'Canada':'North America',
                                    'China':'Asia', 'European Union':'Europe', 'France':'Europe', 'Germany':'Europe', 'India':'Asia', 'Indonesia':'Asia',
                                    'Italy':'Europe', 'Japan':'Asia', 'Mexico':'North America', 'Russia':'Asia', 'Saudi Arabia':'Asia', 'South Africa':'Africa', 
                                    'South Korea':'Asia', 'Turkey':'Asia', 'United Kingdom':'Europe', 'United States':'North America'})

g20_covid['Continent'] = g20_covid['location'].map({'Argentina':'South America', 'Australia':'Ocenia', 'Brazil':'South America', 'Canada':'North America',
                                    'China':'Asia', 'European Union':'Europe', 'France':'Europe', 'Germany':'Europe', 'India':'Asia', 'Indonesia':'Asia',
                                    'Italy':'Europe', 'Japan':'Asia', 'Mexico':'North America', 'Russia':'Asia', 'Saudi Arabia':'Asia', 'South Africa':'Africa', 
                                    'South Korea':'Asia', 'Turkey':'Asia', 'United Kingdom':'Europe', 'United States':'North America'})

C:\Users\kanen\AppData\Local\Temp\ipykernel_13664\2309399557.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kanen\AppData\Local\Temp\ipykernel_13664\2309399557.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kanen\AppData\Local\Temp\ipykernel_13664\2309399557.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [32]:
index_2020 = g20_index_data.loc[(g20_index_data['date'] >= "2020-01-01") & (g20_index_data['date'] <= "2020-12-31")]
index_2020['date'] =  pd.to_datetime(index_2020['date'], infer_datetime_format=True)
index_2020["week"] = index_2020["date"].dt.isocalendar().week
index_2020_week =  index_2020.groupby(["Country", "week"],as_index=False).mean()

C:\Users\kanen\AppData\Local\Temp\ipykernel_13664\667688445.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kanen\AppData\Local\Temp\ipykernel_13664\667688445.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



##### Daily Average of G20 Financial Markets in 2020

In [51]:
g20_index_data= g20_index_data.sort_values("day")
fig = px.choropleth(g20_index_data, locations='Country',locationmode = "country names", color="pct_change",range_color=(-5, 5), color_continuous_scale="YlGnBu", animation_frame="day")
fig.update_layout(title ="Daily Average of G20 Financial Markets in 2020",  margin={"r":0,"t":0,"l":0,"b":0}, transition= {'duration':1})
fig.show()

**Conclusion:** I guess it shows the volatilty in some of the G20 markets while others stayed somewhat steady. Turkey got destroyed 

##### Same Data but in Line Graph Form:

In [45]:
index_2020 = g20_index_data.loc[(g20_index_data['date'] >= "2020-01-01") & (g20_index_data['date'] <= "2020-12-31")]
index_2020['date'] =  pd.to_datetime(index_2020['date'], infer_datetime_format=True)
index_2020["day"] = index_2020["date"].dt.day_of_year
index_2020_day =  index_2020.groupby("day",as_index=False).mean()

C:\Users\kanen\AppData\Local\Temp\ipykernel_13664\2458218675.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kanen\AppData\Local\Temp\ipykernel_13664\2458218675.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
index_2020_day

,day,USD,year,month,return,pct_change
0,1,5688.091078,2020.0,1.0,-8.538342,-1.061690
1,2,5778.704675,2020.0,1.0,90.613596,2.173224
2,3,5727.651622,2020.0,1.0,-51.053053,-0.373381
3,4,5719.889411,2020.0,1.0,-7.762210,-1.009402
4,5,5719.889411,2020.0,1.0,0.000000,0.000000
...,...,...,...,...,...,...
361,362,4557.843260,2020.0,12.0,0.000000,0.000000
362,363,4586.958852,2020.0,12.0,29.115591,1.835201
363,364,4591.349502,2020.0,12.0,4.390650,0.362694
364,365,4598.352125,2020.0,12.0,7.002623,0.269311


In [58]:
fig=px.line(index_2020_day, x="day", y="pct_change")
fig.add_vline(x=51, line_dash="dash")
fig.add_vline(x=96, line_dash="dash")
fig.update_layout(title = "Daily Avg. % Change of G20 Financial Markets in 2020", xaxis_title="Day", yaxis_title = "% Change")
fig.add_annotation(x=40
    , y=4
    , text='Bear'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=20, color="black", family="Sans Serif")
    , align="left"
    ,)
fig.add_annotation(x=105
    , y=4
    , text='Bull'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=20, color="black", family="Sans Serif")
    , align="left"
    ,)
fig.show()

In [ ]:
display(g20_covid)
g20_index_data.columns.values

In [ ]:
g20_covid_subset = g20_covid[["date","location", "new_cases", "new_deaths", "new_tests", "new_vaccinations"]]
g20_index_subset = g20_index_data[["date", "USD"]]
g20_covid_subset['date'] =  pd.to_datetime(g20_covid_subset['date'], infer_datetime_format=True)
g20_index_subset['date'] =  pd.to_datetime(g20_index_subset['date'], infer_datetime_format=True)

##### Correlation Heatmap for all Countries 

In [ ]:
g20_covid_index = g20_covid_subset.merge(g20_index_subset, on='date', how='left')
f, ax1 = plt.subplots(figsize=(20, 16))
ax = sns.heatmap(g20_covid_index.iloc[:, 1:].corr(), annot=True, cmap="YlGnBu")

In [ ]:
g20_covid_canada = g20_covid_subset.loc[g20_covid_subset['location'] == "Canada"]
g20_index_canada = g20_index_data.loc[g20_index_data['Country'] == "Canada"]
g20_covid_canada['date'] =  pd.to_datetime(g20_covid_canada['date'], infer_datetime_format=True)
g20_index_canada['date'] =  pd.to_datetime(g20_index_canada['date'], infer_datetime_format=True)


##### Correlation Heatmap for all Just Canada

In [ ]:
g20_covid_index_canada = g20_covid_canada.merge(g20_index_canada, on='date', how='left')
g20_covid_index_canada = g20_covid_index_canada[["date", "new_cases", "new_deaths", "new_tests", "new_vaccinations", "USD"]]
f, ax1 = plt.subplots(figsize=(20, 16))
ax = sns.heatmap(g20_covid_index_canada.iloc[:, 1:].corr(), annot=True, cmap="YlGnBu")